In [1]:
# Link to Pyhton Hangman AI: https://repl.it/@asamanta/hangmanai


word_list = ["hello", "cello", "grape", "grate",
             "grass", "brass", "right", "sight",
             "maria", "chair", "wooly", "laced",
             "smile", "ghost", "fries", "amaze",
             "sleep", "jewel"]

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

26-element Array{Char,1}:
 'a'
 'b'
 'c'
 'd'
 'e'
 'f'
 'g'
 'h'
 'i'
 'j'
 'k'
 'l'
 'm'
 'n'
 'o'
 'p'
 'q'
 'r'
 's'
 't'
 'u'
 'v'
 'w'
 'x'
 'y'
 'z'

In [3]:
letter_pattern(word, letter) =
    map(char -> char == letter ? char : '-',word)

partition_list(word_list, letter) =
  let d = Dict()
    for word in word_list
      let pattern = letter_pattern(word, letter)
        d[pattern] = push!(get(() -> [], d, pattern), word)
      end
    end
    values(d)
  end

list_of_lengths_partition(word_list, letter) =
    let partitions = partition_list(word_list, letter)
        map(x -> length(x), partitions)
    end

list_of_lengths_partition (generic function with 1 method)

In [4]:
measure_information1(list_lengths) =
    reduce(max, list_lengths)

measure_information2(list_lengths) =
    length(list_lengths)

entropy(list_lengths) =
    let total_length = reduce(+, list_lengths)
        -reduce((total, len) -> total + len / total_length * log2(len / total_length), list_lengths; init=0)
    end

entropy (generic function with 1 method)

In [5]:
function best_letter_internal_bigger(state, letter)
    current_index, best_value, best_letter, word_list, measure = state
    information = measure(list_lengths)
    if information > best_value
        current_index + 1, information, letter, word_list, measure
    else
        current_index + 1, best_value, best_letter, word_list, measure
    end
end

function best_letter_internal_smaller(state, letter)
    current_index, best_value, best_letter, word_list, measure = state
    information = measure(list_lengths)
    if information < best_value
        current_index + 1, information, letter, word_list, measure
    else
        current_index + 1, best_value, best_letter, word_list, measure
    end
end

function best_letter1(word_list, measure, bigger_better)
    if bigger_better == true
        current_index, best_value, best_letter, x, y = reduce(best_letter_internal_bigger, letters, (0, Inf, "dummy", word_list, measure))
    elseif bigger_better == false
        current_index, best_value, best_letter, x, y = reduce(best_letter_internal_smaller, letters, (0, Inf, "dummy", word_list, measure))
    end
end

function best_letter_bob(state, letter)
    current_index, best_value, best_letter, word_list, measure, comparison = state
    information = measure(list_lengths)
    if comparison(information, best_value)
        current_index + 1, information, letter, word_list, measure
    else
        current_index + 1, best_value, best_letter, word_list, measure
    end
end

best_letter_bob (generic function with 1 method)

In [6]:
function word_pattern_match(word, letter, pattern)
    letter_pattern(word, letter) == pattern
end

function matching_words(list_words, letter, pattern)
    result = []
    for word in list_words
        if word_pattern_match(word, letter, pattern) == true
            push!(result, word)
        end
    end
    result
end

function matching_words_filter(list_words, letter, pattern)
    result = filter(word -> word_pattern_match(word, letter, pattern), list_words)
    result
end

matching_words_filter (generic function with 1 method)

In [7]:
println(letter_pattern("hello", 'l'))
list_lengths = list_of_lengths_partition(word_list, 'e')
println(list_lengths)
println(measure_information1(list_lengths))
println(measure_information2(list_lengths))
println(entropy(list_lengths))

--ll-
[4, 2, 1, 2, 1, 8]
8
6
2.169925001442312


In [8]:
@assert matching_words(["hello", "jello", "cello"], 'e', "-e---") == ["hello","jello","cello"]
@assert word_pattern_match("hello", 'l', "---l-") == false
@assert matching_words_filter(["hello", "jello", "jewel"], 'l', "--ll-") == ["hello", "jello"]